In [ ]:
import gc
from collections import namedtuple
import xml.etree.ElementTree as ET

import matplotlib.pyplot as plt
import numpy as np

%matplotlib notebook

## Benchmarking on P1 and P2 sets

Start from instrumenting us with functions for appropriate work with CAAP sets and scripts (linking Alingstein output with CAAP evaluation script input etc.).

In [ ]:
Feature = namedtuple('Feature', ['int', 'rt', "mz"])

def parse_feature_xml(filename):
    features = {}
    tree = ET.parse(filename)
    root = tree.getroot()
    feature_list = root.find("./featureList")
    f_numb = int(feature_list.attrib["count"])
    for feature in feature_list.findall("./feature"):
        positions = {int(position.attrib["dim"]):position.text for position in feature.findall("./position")}
        # Mapping with id will further work because features are numbered from 0
        features[feature.attrib["id"]] = Feature(
            int(float(feature.find("./intensity").text)),
            # to avoid rounding problems, keep floats as str
            positions[0], positions[1])
    return f_numb, features
    

def dump_consensus_features_to_caap_file(
    consensus_features, out_filename, chromatograms_sets_list, feature_filenames):
    all_caap_features = []
    for fname in feature_filenames:
        _, caap_features = parse_feature_xml(fname)
        all_caap_features.append(caap_features)
    rows = []
    with open(out_filename, "w") as outfile:
        for consensus_feature in consensus_features:
            if len(consensus_feature) > 1:
                row = []
                for set_i, chromatogram_j in consensus_feature:
                    caap_id = chromatograms_sets_list[set_i][chromatogram_j].ext_id
                    # Yeah, str below, one of many legacy...
                    caap_feature = all_caap_features[set_i][str(caap_id)]
                    row.extend([caap_feature.int, caap_feature.rt, caap_feature.mz])
                rows.append(" ".join(map(str, row)))
        outfile.write("\n".join(rows))

Start from downloading datasets from CAAP webpage: [https://msbi.ipb-halle.de/msbi/caap](https://msbi.ipb-halle.de/msbi/caap). Create `benchmark_datasets`, download data there and unpack them:

```bash
mkdir benchmark_datasets
cd benchmark_datasets
mkdir P1_set
cd P1_set
wget https://msbi.ipb-halle.de/download/P1_features.tar.gz
tar xzf P1_features.tar.gz
wget http://msbi.ipb-halle.de/download/P1-raw.tar.bz2
tar xjf P1-raw.tar.bz2
wget https://msbi.ipb-halle.de/download/P1_ground_truth.tar.gz
tar xzf P1_ground_truth.tar.gz
mkdir ../P2_set
cd ../P2_set
wget https://msbi.ipb-halle.de/download/P2_features.tar.gz
tar xzf P2_features.tar.gz
wget http://msbi.ipb-halle.de/download/P2-raw.tar.bz2
tar xjf P2-raw.tar.bz2
wget https://msbi.ipb-halle.de/download/P2_ground_truth.tar.gz
tar xzf P2_ground_truth.tar.gz
```
Finally, download evaluation script:
```bash
wget https://msbi.ipb-halle.de/download/eval.R
```

Except for the P1 and P2 sets, the CAAP study consisted also of the analysis of two metabolomic (M1 and M2) datasets.
We, omitted, however, reporting these datasets analysis. M2 dataset availability is currently limited and the M1 dataset has feature representation incompatible with Alignstein's features so the results would be inconclusive.

For every set, benchmarking is performed in separate salt bump. We prepared set of filenames for every set of data. Feel free to uncomment interesting part.

In [ ]:
data_filenames = [
    "benchmark_datasets/P1_set/P1/021010_jp32A_15ul_1_000_ld_020.mzXML",
    "benchmark_datasets/P1_set/P1/021016_jp32A_10ul_3_000_ld_020.mzXML"]
feature_filenames = [
    "benchmark_datasets/P1_set/P1_features/021010_jp32A_15ul_1_000_ld_020.featureXML",
    "benchmark_datasets/P1_set/P1_features/021016_jp32A_10ul_3_000_ld_020.featureXML"]
gt_filename = "benchmark_datasets/P1_set/P1_ground_truth/ground_truth_Ecoli_000_ld_020.dat"

# data_filenames = [
#     "benchmark_datasets/P1_set/P1/021010_jp32A_15ul_1_020_ld_040.mzXML",
#     "benchmark_datasets/P1_set/P1/021016_jp32A_10ul_3_020_ld_040.mzXML"]
# feature_filenames = [
#     "benchmark_datasets/P1_set/P1_features/021010_jp32A_15ul_1_020_ld_040.featureXML",
#     "benchmark_datasets/P1_set/P1_features/021016_jp32A_10ul_3_020_ld_040.featureXML"]
# gt_filename = "benchmark_datasets/P1_set/P1_ground_truth/ground_truth_Ecoli_020_ld_040.dat"

# data_filenames = [
#     "benchmark_datasets/P1_set/P1/021010_jp32A_15ul_1_040_ld_060.mzXML",
#     "benchmark_datasets/P1_set/P1/021016_jp32A_10ul_3_040_ld_060.mzXML"]
# feature_filenames = [
#     "benchmark_datasets/P1_set/P1_features/021010_jp32A_15ul_1_040_ld_060.featureXML",
#     "benchmark_datasets/P1_set/P1_features/021016_jp32A_10ul_3_040_ld_060.featureXML"]
# gt_filename = "benchmark_datasets/P1_set/P1_ground_truth/ground_truth_Ecoli_040_ld_060.dat"

# data_filenames = [
#     "benchmark_datasets/P1_set/P1/021010_jp32A_15ul_1_060_ld_080.mzXML",
#     "benchmark_datasets/P1_set/P1/021016_jp32A_10ul_3_060_ld_080.mzXML"]
# feature_filenames = [
#     "benchmark_datasets/P1_set/P1_features/021010_jp32A_15ul_1_060_ld_080.featureXML",
#     "benchmark_datasets/P1_set/P1_features/021016_jp32A_10ul_3_060_ld_080.featureXML"]
# gt_filename = "benchmark_datasets/P1_set/P1_ground_truth/ground_truth_Ecoli_060_ld_080.dat"

# data_filenames = [
#     "benchmark_datasets/P1_set/P1/021010_jp32A_15ul_1_080_ld_100.mzXML",
#     "benchmark_datasets/P1_set/P1/021016_jp32A_10ul_3_080_ld_100.mzXML"]
# feature_filenames = [
#     "benchmark_datasets/P1_set/P1_features/021010_jp32A_15ul_1_080_ld_100.featureXML",
#     "benchmark_datasets/P1_set/P1_features/021016_jp32A_10ul_3_080_ld_100.featureXML"]
# gt_filename = "benchmark_datasets/P1_set/P1_ground_truth/ground_truth_Ecoli_080_ld_100.dat"

# data_filenames = [
#     "benchmark_datasets/P1_set/P1/021010_jp32A_15ul_1_100_ld_150.mzXML",
#     "benchmark_datasets/P1_set/P1/021016_jp32A_10ul_3_100_ld_150.mzXML"]
# feature_filenames = [
#     "benchmark_datasets/P1_set/P1_features/021010_jp32A_15ul_1_100_ld_150.featureXML",
#     "benchmark_datasets/P1_set/P1_features/021016_jp32A_10ul_3_100_ld_150.featureXML"]
# gt_filename = "benchmark_datasets/P1_set/P1_ground_truth/ground_truth_Ecoli_100_ld_150.dat"

# data_filenames = [
#     "benchmark_datasets/P2_set/P2/7-17-03_000.mzXML",
#     "benchmark_datasets/P2_set/P2/6-17-03_000.mzXML",
#     "benchmark_datasets/P2_set/P2/6-06-03_000.mzXML"]
# feature_filenames = [
#     "benchmark_datasets/P2_set/P2_features/7-17-03_000.featureXML",
#     "benchmark_datasets/P2_set/P2_features/6-17-03_000.featureXML",
#     "benchmark_datasets/P2_set/P2_features/6-06-03_000.featureXML"]
# gt_filename = "benchmark_datasets/P2_set/P2_ground_truth/ground_truth_Msmeg_000.dat"

# data_filenames = [
#     "benchmark_datasets/P2_set/P2/7-17-03_020.mzXML",
#     "benchmark_datasets/P2_set/P2/6-17-03_020.mzXML",
#     "benchmark_datasets/P2_set/P2/6-06-03_020.mzXML"]
# feature_filenames = [
#     "benchmark_datasets/P2_set/P2_features/7-17-03_020.featureXML",
#     "benchmark_datasets/P2_set/P2_features/6-17-03_020.featureXML",
#     "benchmark_datasets/P2_set/P2_features/6-06-03_020.featureXML"]
# gt_filename = "benchmark_datasets/P2_set/P2_ground_truth/ground_truth_Msmeg_020.dat"

# data_filenames = [
#     "benchmark_datasets/P2_set/P2/6-06-03_040.mzXML",
#     "benchmark_datasets/P2_set/P2/6-17-03_040.mzXML",
#     "benchmark_datasets/P2_set/P2/7-17-03_040.mzXML"]
# feature_filenames = [
#     "benchmark_datasets/P2_set/P2_features/6-06-03_040.featureXML",
#     "benchmark_datasets/P2_set/P2_features/6-17-03_040.featureXML",
#     "benchmark_datasets/P2_set/P2_features/7-17-03_040.featureXML"]
# gt_filename = "benchmark_datasets/P2_set/P2_ground_truth/ground_truth_Msmeg_040.dat"

# data_filenames = [
#     "benchmark_datasets/P2_set/P2/6-06-03_080.mzXML",
#     "benchmark_datasets/P2_set/P2/6-17-03_080.mzXML",
#     "benchmark_datasets/P2_set/P2/7-17-03_080.mzXML"]
# feature_filenames = [
#     "benchmark_datasets/P2_set/P2_features/6-06-03_080.featureXML",
#     "benchmark_datasets/P2_set/P2_features/6-17-03_080.featureXML",
#     "benchmark_datasets/P2_set/P2_features/7-17-03_080.featureXML"]
# gt_filename = "benchmark_datasets/P2_set/P2_ground_truth/ground_truth_Msmeg_080.dat"

# data_filenames = [
#     "benchmark_datasets/P2_set/P2/6-06-03_100.mzXML",
#     "benchmark_datasets/P2_set/P2/6-17-03_100.mzXML",
#     "benchmark_datasets/P2_set/P2/7-17-03_100.mzXML"]
# feature_filenames = [
#     "benchmark_datasets/P2_set/P2_features/7-17-03_100.featureXML",
#     "benchmark_datasets/P2_set/P2_features/6-17-03_100.featureXML",
#     "benchmark_datasets/P2_set/P2_features/6-06-03_100.featureXML"]
# gt_filename = "benchmark_datasets/P2_set/P2_ground_truth/ground_truth_Msmeg_100.dat"

Parse feature file, detect feature signal in raw chromatograms.

In [ ]:
# Feature parsing and detection
from Alignstein import (parse_chromatogram_with_detected_features,
                        features_to_weight)

feature_sets_list = [
    parse_chromatogram_with_detected_features(ch_fname, fs_fname)
        for ch_fname, fs_fname in zip(data_filenames, feature_filenames)]

In [ ]:
# Scale by average weight
SCALE_FACTOR = 10 # value chosen as explained in tutorial

weights = [features_to_weight(f_set) for f_set in feature_sets_list]
average_weight = np.mean(weights)

print("Weights:", weights, "\n", "Average weight", average_weight)

for feature_set in feature_sets_list:
    for feature in feature_set:
        feature.scale_rt(average_weight * SCALE_FACTOR)

In [ ]:
# Load ground-truth

gts = set()
with open(gt_filename, "r") as infile:
    for line in infile:
        line = line.strip().split(" ")
        if len(line) > 10:
            gts.add(((line[2], line[3], line[4]), (line[7], line[8], line[9]), (line[7+5], line[8+5], line[9+5])))
        else:
            gts.add(((line[2], line[3], line[4]), (line[7], line[8], line[9])))

Originally, in the CAAP study, only a fraction of all detected features were aligned. The evaluation protocol lacks, however, a detailed description of initial feature filtering for further alignment. Thus, we decided to filter features to those existing in ground-truth analogously as reported in Wandy et al. [1].

In [ ]:
feature_sets_list_old = feature_sets_list

selected_features = {t[0] for t in gts}.union({t[1] for t in gts}).union({t[2] for t in gts if len(t) > 2})
selected_feature_sets_list = []

for i, feature_set in enumerate(feature_sets_list):
    _, caap_features = parse_feature_xml(feature_filenames[i])
    selected_f_set = []
    for ch in feature_set:
        caap_id = ch.ext_id
        caap_feature = caap_features[str(caap_id)] # It works because caap features are enumerated from 0.
        t = (str(float(caap_feature.int)), caap_feature.rt, caap_feature.mz)
        if t in selected_features:
            selected_f_set.append(ch)
    selected_feature_sets_list.append(selected_f_set)

feature_sets_list = selected_feature_sets_list

Do the alignment, after importing Alingstein's functions, first cell contains alignment for P1 (pairwise), next cell contain alignment for P2 (multialignment).

In [ ]:
from Alignstein import (gather_mids, precluster_mids, big_clusters_to_clusters,
                        find_consensus_features,
                        find_pairwise_consensus_features)

In [ ]:
# Pairwise alignment (P1 example)

consensus_features, matched_all_sets = find_pairwise_consensus_features(
    *feature_sets_list, centroid_upper_bound=20, gwd_upper_bound=20,
    matching_penalty=10)

dump_consensus_features_to_caap_file(
    consensus_features,
    "examples/LCMS_data/benchmark_datasets/P1_set/alignstein.out",
    feature_sets_list, feature_filenames)

In [ ]:
# Multialignment (P2 example)
mids = gather_mids(feature_sets_list)
gc.collect()
big_clusters = precluster_mids(mids)

# Distance threshold for clusters should be about 2 x max M/Z difference    
clusters = big_clusters_to_clusters(mids, big_clusters,
                                    distance_threshold=1.5)

consensus_features = find_consensus_features(
    clusters, feature_sets_list,
    centroid_upper_bound=10, gwd_upper_bound=10,
    matching_penalty=10)

dump_consensus_features_to_caap_file(
    consensus_features,
    "examples/LCMS_data/benchmark_datasets/P2_set/alignstein.out",
    feature_sets_list, feature_filenames)

For next part contains analysis of `alignstein.out` file by evaluation script provided by authors of CAAP study. Goto `benchmark_datasets` directory, open R console and type:
```R
gt <- 'P1_set/P1_ground_truth/ground_truth_Ecoli_000_ld_020.dat' # Replace with appropriate GT filename
tool <- 'alignstein.out'
eval(gt, tool)
```

## Additional comments
### Additional comments to CAAP evaluation results
OpenMS alignment algorithm performed best in the CAAP study. Originally, the authors of this study evaluated the OpenMS version 1.0. Its alignment algorithm was reimplemented in 2012 and the previous version is no longer bundled with the OpenMS package. We reproduced the evaluation of the CAAP study on the current version of OpenMS. Unfortunately, the current alignment algorithm is achieving significantly worse results despite strenuous attempts to adjust the algorithm parameters to the data. Its alignment precision and recall are on average 60 percentage points lower than the results reported in the CAAP study.

The majority of alignment algorithms are not compared with any tool [2]. This results in difficulties in broad comparing Alignstein with the majority of algorithms. Thus, there is a constant need for dedicated LC-MS alignment assessment of currently being state-of-the-art alignment software that not only complements CAAP with other datasets but also selects the best currently available alignment algorithm.
To the best of the authors’ knowledge, CAAP is the only assessment of LC-MS alignment algorithms done on real datasets and thus it is widely used for validation.
The limited availability of benchmark datasets may result in a growing tendency to analyze algorithms only on data from CAAP work and, therefore, to overfit to this dataset.

## References:
[1] Joe Wandy, Rónán Daly, Rainer Breitling, Simon Rogers, Incorporating peak grouping information for alignment of multiple liquid chromatography-mass spectrometry datasets, Bioinformatics, Volume 31, Issue 12, 15 June 2015, Pages 1999–2006, https://doi.org/10.1093/bioinformatics/btv072

[2] Robert Smith, Dan Ventura, John T. Prince, Novel algorithms and the benefits of comparative validation, Bioinformatics, Volume 29, Issue 12, 15 June 2013, Pages 1583–1585, https://doi.org/10.1093/bioinformatics/btt176